Tutorial for MMD with the TorchDrift library: https://towardsai.net/p/machine-learning/drift-detection-using-torchdrift-for-tabular-and-time-series-data

more documentation on TorchDrift MMD: https://torchdrift.org/notebooks/note_on_mmd.html

In [4]:
import pandas as pd
import torch
import numpy as np
import os
import torchdrift.detectors as detectors

##### On the (Statistical) Detection of Adversarial Examples

**Two-sample hypothesis testing** — As stated before, the test we chose is appropriate to handle high dimensional inputs and small sample sizes. We compute the biased estimate of MMD using a **Gaussian kernel**, and then apply **10 000 bootstrapping iterations** to estimate the distributions. Based on this, we compute the **pvalue** and compare it to the threshold, in our experiments **0.05**. For samples of **legitimate data, the observed p-value should always be very high**, whereas for sample sets containing adversarial examples, we expect it to be low—since they are sampled from a different distribution and thus the hypothesis should be rejected. The test is more likely to detect a difference in two distributions when it considers samples of large size (i.e., the sample contains more inputs from the distribution).

In [5]:
BOOTSTRAP = 10_000
PVAL = 0.05
kernel = detectors.mmd.GaussianKernel()

Because our dataset is a time series, we will use MMD on different time segments rather than shuffling the dataset

Load unperturbed observations from untargeted adversarial attack

In [6]:
df_adv_obs = pd.read_csv('optimal BB observations.csv', 
                        index_col=0,
                        dtype='float32')
df_adv_obs.set_index(df_adv_obs.index.astype(int), inplace=True) #all data is loaded as float32, but the index should be an int

Load perturbed observations from untargeted adversarial attack (100% adversarial)

In [7]:
df_adv_perturbed_obs = pd.read_csv('optimal BB adversarial observations.csv', 
                        index_col=0,
                        dtype='float32')
df_adv_perturbed_obs.set_index(df_adv_perturbed_obs.index.astype(int), inplace=True) 

Here we'll get the MMD between two full distrubtions during evaluation, the observations from the environment and the same observations once perturbed by BB

In [8]:
result = detectors.kernel_mmd(torch.from_numpy(df_adv_obs.values).to('cuda'), #clean obs from adv trace
                                  torch.from_numpy(df_adv_perturbed_obs.values).to('cuda'), #perturbed obs from adv trace
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
print(f'mmd:{result[0]}, p-value:{result[1]}')

mmd:0.06381118297576904, p-value:0.0


Determin the minimal amount of adversarial samples required for detection using a binary search

In [13]:
min = 0
max = df_adv_perturbed_obs.shape[0]
#only transfer data to gpu once
clean_obs = torch.from_numpy(df_adv_obs.values).to('cuda')
adv_obs = torch.from_numpy(df_adv_perturbed_obs.values).to('cuda')

while True:
    if min < max:
        mid = (max - min)//2
    else:
         break
    mmd, pval = detectors.kernel_mmd(clean_obs[:mid], #clean obs from adv trace
                                    adv_obs[:mid], #perturbed obs from adv trace
                                    n_perm=BOOTSTRAP,
                                    kernel=kernel)
    if pval > PVAL: #distributions are the same, more samples needed to find difference
        min = mid
    else: #distributions are different, will fewer samples detect the perturbations?
        max = mid

print(f'With a threshold of {PVAL}, more than {mid} samples are required to detect the adversarial perturbations.\n MMD: {mmd}, p-value: {pval}')

With a threshold of 0.05, at minimum 8 samples are required to detect the adversarial perturbations.
 MMD: 0.38517332077026367, p-value: 0.033900000154972076


We can  make this test more robust by testing slices starting elsewhere